# CNN

In [1]:
import sys
sys.path.append('/notebooks/MOD2NN')

In [49]:
import os
import json

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

from CNN.module.lib.layers import *
from Faraday.two_dim.module.lib.layers import *
tf.random.set_seed(1)

print("TensorFlow:", tf.__version__)
print("Python:", sys.version)

plt.rcParams['font.size'] = 18

TensorFlow: 2.8.0
Python: 3.8.11 (default, Aug 16 2021, 12:04:33) 
[Clang 12.0.0 ]


In [48]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.sqrt(x_train.astype("float32") / 255.0)
x_test = np.sqrt(x_test.astype("float32") / 255.0)
y_train_onehot = tf.one_hot(y_train, 10)

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")
y_test_onehot = tf.one_hot(y_test, 10)

# Reserve 10,000 samples for validation
x_val = x_train[-5000:]
y_val = y_train[-5000:]
x_train = x_train[:-5000]
y_train = y_train[:-5000]

In [43]:
def create_model(shape):
    inputs = tf.keras.Input((28,28))
    x = ImageResizing(inputs)
    x = layers.Flatten()(x)
    x = Real2Complex()(x)
    x = ComplexDense(10)(x)
    x = MulConjugate()(x)
    x = Real()(x)

    return tf.keras.Model(inputs, x)

In [34]:
def train(i,seed):
    shape = (28*i, 28*i)
    tf.random.set_seed(seed)
    model_name = "20221124_{:d}_seed_{:d}_shape({:d}_{:d})".format(i, seed, *shape)
    path = "../../"
    checkpoint_path = path + "checkpoint/" + model_name + "/cp-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    path = path + "trained_model/"+ model_name

    model = create_model((28*i, 28*i))
    model.summary()
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),  # category: sparse_categorical_crossentropy
                  metrics=[tf.keras.metrics.BinaryCrossentropy(from_logits=True)]
                  )

    epochs = 50
    early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='loss',
            min_delta=0.0002,
            patience=2,
        )

    def lr_schedule(epoch):
      """
      Returns a custom learning rate that decreases as epochs progress.
      """
      start = 0.2
      end = 0.0001
      n = 50
      a = -1/(n-1) * np.log(end/start)
      x = np.arange(n)
      return np.exp(-a*(epoch-1)) * start
    lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

    # チェックポイントコールバックを作る
    cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=1)

    logdir = os.path.join(path +"logs", model_name)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    result = model.fit(x_train,
                       y_train_onehot,
                       batch_size=64,
                       epochs=epochs,
                       validation_data=(x_val, y_val),
                       callbacks=[cp_callback]
                       )

    model.save(path)

    df = pd.DataFrame(result.history)
    df.to_csv(path + "/history.csv")



    with open(path + "/config.json", 'w') as f:
        json.dump(model.get_config(), f, indent=4)

In [ ]:
index = [1, 2, 3]
seeds = [1,2,3,4,5]

for i in index:
    for seed in seeds:
        train(i, seed)